In [11]:
import pandas as pd
import numpy as np
import sklearn
import os
from tqdm import tqdm, trange, tqdm_notebook
import re
import nltk
from multiprocessing import cpu_count, Pool
from functools import partial
import ipywidgets
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
X_train = pd.read_hdf("X_train_full_allfeatures-NOLABEL.h5", key="df")
y_train = pd.read_hdf("y_train_full.h5", key="df")

In [3]:
X_train.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,police find mass graves with at least 15 bodie...,"[police, find, mass, graf, with, at, least, 15...",danny boyle is directing the untitled film set...,"[danny, boyle, is, directing, the, untitled, f...",2,0,0,0.014085,...,0,0,0,0,0,0,0,0,0,0
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,hundreds of palestinians flee floods in gaza a...,"[hundred, of, palestinian, flee, flood, in, ga...",hundreds of palestinians were evacuated from t...,"[hundred, of, palestinian, were, evacuated, fr...",10,7,7,0.046083,...,3,0,0,0,0,0,0,0,0,0
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,christian bale passes on role of steve jobs ac...,"[christian, bale, pass, on, role, of, steve, j...",30 year old moscow resident was hospitalized w...,"[30, year, old, moscow, resident, wa, hospital...",5,4,1,0.030303,...,1,0,0,0,0,0,0,0,0,0
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,hbo and apple in talks for 15 month apple tv s...,"[hbo, and, apple, in, talk, for, 15, month, ap...",reuters a canadian soldier was shot at the can...,"[reuters, a, canadian, soldier, wa, shot, at, ...",3,3,0,0.028169,...,1,0,0,0,0,0,0,0,0,0
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",spider burrowed through tourist s stomach and ...,"[spider, burrowed, through, tourist, s, stomac...",fear not arachnophobes the story of bunbury s ...,"[fear, not, arachnophobes, the, story, of, bun...",9,5,4,0.032727,...,4,0,0,0,0,0,0,0,0,0


In [4]:
y_train.head()

0    unrelated
1        agree
2    unrelated
3    unrelated
4     disagree
Name: Stance, dtype: object

In [5]:
print(X_train.shape)
print(y_train.shape)

(49972, 49)
(49972,)


In [6]:
del X_train["articleBody"]
del X_train["Headline"]
for col_name in X_train.columns:
    if "___" == col_name[0:3]:
        del X_train[col_name]

In [7]:
X_train.head()

,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,wrf_hl_fake,wrf_hl_fraud,wrf_hl_hoax,wrf_hl_false,wrf_hl_deny,wrf_hl_denies,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
0,2,0,0,0.014085,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,7,7,0.046083,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
2,5,4,1,0.030303,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,3,3,0,0.028169,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,9,5,4,0.032727,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0


In [8]:
X_train, X_dev, y_train, y_dev = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_dev.shape)
print(y_dev.shape)

(44974, 43)
(44974,)
(4998, 43)
(4998,)


In [10]:
X_train.head()

,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,wrf_hl_fake,wrf_hl_fraud,wrf_hl_hoax,wrf_hl_false,wrf_hl_deny,wrf_hl_denies,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
44610,9,8,4,0.057377,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
10499,4,3,1,0.023256,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
43957,8,5,4,0.024465,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
49559,4,2,1,0.009926,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1851,1,1,0,0.008403,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [12]:
clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)

In [13]:
clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1       44003.6782           44.82s
         2       39386.2805           42.36s
         3       35647.5327           41.60s
         4       32591.8800           43.49s
         5       30074.0478           43.13s
         6       27969.2272           41.99s
         7       26212.1162           41.59s
         8       24722.2751           42.17s
         9       23476.4802           41.41s
        10       22386.3081           41.45s
        20       17304.8224           37.61s
        30       15906.0471           36.34s
        40       15296.5875           33.05s
        50       14989.3720           29.87s
        60       14763.3830           27.56s
        70       14602.0671           25.17s
        80       14457.9762           22.97s
        90       14333.8437           20.82s
       100       14229.1701           19.00s
       200       13507.8045            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=14128, subsample=1.0,
              verbose=True, warm_start=False)

In [14]:
preds = clf.predict(X_dev)

In [16]:
actual = y_dev

In [17]:
from sklearn.metrics import classification_report

In [19]:
# I checked this against the baseline implementation -- numbers look correct!
print(classification_report(actual, preds))

             precision    recall  f1-score   support

      agree       0.55      0.16      0.24       331
   disagree       0.40      0.02      0.04       102
    discuss       0.65      0.84      0.73       876
  unrelated       0.96      0.98      0.97      3689

avg / total       0.87      0.88      0.86      4998

